# Finding a reliable betting strategy in the German Bundesliga for the next Season

This notebook was first published on my blog [www.bersten.xyz](www.bersten.xyz). I want to thank those who proof read the first draft and gave me inspiration and constructive critique.

**Disclaimer:** *In this article I will describe how to do a technical analysis of the football betting market. This approach will help to find possible profitable scenarios in the said betting market. Be aware that football bets are considered **gambling**. In no way, I give any financial advise, nor am I your financial advisor. There is risk involved and you can lose money applying the strategy described below.*


Now let's begin: This notebook is a *quasi excerpt* from my BA thesis, I wrote a few years ago. I tested the European Football Betting Market against the background of the *"Efficient Market-Hypothesis"*. For simplification purposes, I will only focus on the German 1. Bundesliga for now. The structure of the notebook is as follows:

1. Introduction
2. We will have a look at the dataset
3. A bit a data wrangling
4. Strategy Check: Arbitrage
5. Strategy Check: Common betting strategies
6. Conclusion


## Data origin

The dataset is available at [football-data.co.uk](http://www.football-data.co.uk/data.php). I only used the data for German Bundesliga but football-data.co.uk cover much more leagues. You can find the notes with explanations for the data columns [here](http://www.football-data.co.uk/notes.txt). I will just post the list to map the bookmakers to their column abbreviations.

List with keys to 1X2 (match) betting odds data:

```
B365H = Bet365 home win odds
B365D = Bet365 draw odds
B365A = Bet365 away win odds

BWH = Bet&Win home win odds
BWD = Bet&Win draw odds
BWA = Bet&Win away win odds

GBH = Gamebookers home win odds
GBD = Gamebookers draw odds
GBA = Gamebookers away win odds

IWH = Interwetten home win odds
IWD = Interwetten draw odds
IWA = Interwetten away win odds

LBH = Ladbrokes home win odds
LBD = Ladbrokes draw odds
LBA = Ladbrokes away win odds

PSH and PH = Pinnacle home win odds
PSD and PD = Pinnacle draw odds
PSA and PA = Pinnacle away win odds

VCH = VC Bet home win odds
VCD = VC Bet draw odds
VCA = VC Bet away win odds

WHH = William Hill home win odds
WHD = William Hill draw odds
WHA = William Hill away win odds
```

# Introduction to the dataset: Getting the data

As I mentioned above, I only downloaded the spreadsheets for the German 1. Bundesliga for the last 3 years and will read it into a pandas dataframe. I wrote a small wrapper function to load the data. We will also set the date column as index. For none-programmers we will now look at a bit of code and will explain the outcome afterwards.

In [1]:
# Impor libs
import os
import pandas as pd
import numpy as np

In [2]:
# print lib versions
print("numpy: {}\npandas: {}\n".format(pd.__version__, np.__version__))

numpy: 0.25.0
pandas: 1.15.0



In [3]:
def load_data():
    """
    Function to read all files from football-data.co.uk in the ./data/ folder.
    Returns: pandas dataframe with date as index
    """
    df = pd.DataFrame()
    files = os.listdir("./data/")
    for file in files:
        _ = pd.read_csv("./data/"+file)
        df = pd.concat([df, _], sort=False)
    # Convert date and set index
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", inplace=True)
    
    return df

In [4]:
# First, load the data with the defined function and look at a random sample of 5 rows
df = load_data()
df.sample(5)

,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-20,D1,Hamburg,Dortmund,0,3,A,0,1,A,6,...,2.46,16,1.00,2.29,2.23,1.71,1.68,8.57,5.69,1.37
2019-05-18,D1,Hertha,Leverkusen,1,5,A,1,2,A,9,...,3.57,20,1.00,2.31,2.24,1.72,1.69,6.07,5.21,1.48
2018-09-29,D1,Hoffenheim,RB Leipzig,1,2,A,0,0,D,13,...,2.70,20,-0.25,1.84,1.80,2.14,2.07,2.27,3.89,3.05
2019-03-16,D1,Stuttgart,Hoffenheim,1,1,D,0,1,A,13,...,2.76,20,0.50,2.02,1.97,1.95,1.89,3.85,4.05,1.92
2019-02-16,D1,Hoffenheim,Hannover,3,0,H,2,0,H,26,...,3.34,19,-1.50,1.87,1.81,2.11,2.05,1.26,6.79,10.50


What do we see here? The data is displayed as a *pandas dataframe*, which follows a basic heterogeneous tabular display format with two dimensions: columns and rows. You can also call one row an *observation*. In our context, one observation resembles one football match-up. The columns contain additional data, here we have 63 columns. The column label on top describes the kind of additional information. The meaning of *HomeTeam* and *AwayTeam* is obvious. For instance, *FTR* is short for *full time result*. You can find the explanation of column labels in the data source.


# Data wrangling and manipulation

In the next step, we will do some data wrangling and manipulation. To begin with, we will look at some of the available columns. As wrote above, the notes are available [here](http://www.football-data.co.uk/notes.txt). Afterwards we will add a *Season* column which will be mapped to the corresponding games. Note that in the German Bundesliga there are 18 teams present, playing against each other as home and away team. This results in: $$ 18 \text{ teams} * 17 \text{ match-ups } = 306 \text{ observations per season}$$
This is why the slicing is done in 306 intervals.

In [5]:
# Show the columns
df.columns

Index(['Div', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
       'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
       'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD',
       'WHA', 'VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD',
       'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5',
       'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA',
       'PSCH', 'PSCD', 'PSCA'],
      dtype='object')

In [6]:
# Map Season by positional index slicing
df.loc[:306, 'Season'] = "2016/2017"
df.loc[307:612, 'Season'] = "2017/2018"
df.loc[612:, 'Season'] = "2018/2019"

In [7]:
# Select only columns with odd data
use_cols = ['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
       'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD',
       'WHA', 'VCH', 'VCD', 'VCA']

df_odds = df[use_cols]
df_odds.head()

,HomeTeam,AwayTeam,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,...,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-26,Bayern Munich,Werder Bremen,1.10,10.00,23.00,1.10,9.75,18.50,1.15,7.0,...,26.00,1.11,12.20,27.00,1.11,8.0,21.00,1.12,10.0,22.0
2016-08-27,Augsburg,Wolfsburg,2.80,3.25,2.60,2.80,3.40,2.40,2.60,3.2,...,2.50,2.85,3.43,2.64,2.70,3.2,2.45,2.75,3.4,2.6
2016-08-27,Dortmund,Mainz,1.36,5.00,8.50,1.36,4.60,8.25,1.40,4.4,...,9.00,1.37,5.41,9.49,1.36,4.5,7.00,1.33,5.4,9.0
2016-08-27,Ein Frankfurt,Schalke 04,3.40,3.30,2.20,3.00,3.50,2.20,3.10,3.3,...,2.15,3.39,3.64,2.22,2.88,3.5,2.20,3.25,3.6,2.2
2016-08-27,FC Koln,Darmstadt,1.80,3.50,4.75,1.75,3.50,4.75,1.80,3.5,...,5.00,1.78,3.72,5.31,1.80,3.4,4.00,1.80,3.6,4.8


The data frame above contains the match-up and the available odd data from the different bookmakers. This makes things easier for the following analysis.

# Strategy #1: Arbitrage

## Theory 
This strategy takes advantage of different odds offered by different bookmakers. You are looking for the highest odds offered by different bookmakers. You can calculate the margin $M$ for one bookmaker by taking the sum of the inverted odds:

$$ M = \sum^{}_{i=H,D,A} \frac{1}{q_i} $$

This margin should always be over 1 for any specific bookmaker. But we will take advantage of having accounts at a various number of bookmakers. This will increase the chance to find possible differences in odds, which can be used for arbitrage. The idea is quite simple:

Given a list of bookmakers $J$, we can scan for every outcome and only take the highest possible odd $q_{jk}$ for each outcome $k=H,D,A$. For a margin smaller than one, we can calculate the (risk-free) return as one minus the *aribtrage opportunity margin*. The strategy is then to bet on all outcomes. So in return, you will win regardless of the match's outcome.

A short algorithm is given below:
![finding arbitrage opportunities](./img/algo.png)

The next logical question to be answered is: If I am betting on all outcomes, how should I distribute my wager? The answer can be given mathematically. It is shown quite easily, that you should distribute your wager proportionally to the *aribtrage opportunity margin* $M_s$:

$$
s_k = \frac{1}{M_s * \max^{}_{j \in J} q_{jk}} \quad \text{for all outcomes } k=H,D,A
$$


## Example

Let's look at an real life example, the match *FC Schalke* vs *Hamburg SV* in the season 2013/14: Bookmakers A and B offered the following odds for this match with the three outcomes Home win **(H)**, Away win **(A)** or the game is draw **(D)**:

|_                    |Home Win     |Draw         |Away Win    |Margin    |
|--------------------:|:-----------:|:-----------:|:-----------:|:-----:   |
|Bookmaker A          |1.40         |**4.86**     |6.00         |108.67%   |
|Bookmaker B          |**1.60**     |3.50         |**7.20**     |104.96%   |
|Arbitrage Opportunity|1.60         |4.86         |7.20         |**96.97%**|

This yields a (risk-free) return of **3.03%**. The distribution of your wager is given as:

|_                    |Home Win     |Draw         |Away Win     |
|--------------------:|:-----------:|:-----------:|:-----------:|
|Distribution of wager|64.46%       |21.22%       |14.32%       |



## Okay, but why?

If we find arbitrage opportunities in the historic closing odds (i.e. the odds that were set by the bookmakers right before the match started), it indicates that the betting market in Germany is not efficient (see [Efficient Market Hypotheses](https://en.wikipedia.org/wiki/Efficient-market_hypothesis)). This basically tells us, that the odds are not priced correctly, which in return can be abused.

To be fair it should be mentioned, that the most arbitrage opportunities arise not within the closing odds, but rather live or in the price finding phase (read: bookmaker calculating his odds). Odds can be quite volatile before the match begins. This is often the case when the teams release their today's line-up or some other news hitting the market (e.g. important players miss the game). In addition, the dataset contains only data for seven different bookmakers. It is quite obvious, that the chance for arbitrage increases with the number of bookmakers you look at.

For more in-depth information on this topic I really recommend reading [this article](https://en.wikipedia.org/wiki/Arbitrage_betting) on wikipedia.

## Understood, what about the Bundesliga now?

We will look at the opportunities the following way. First, we will apply the algorithm from the picture above. Then we will concatenate a dataframe with all with the highest odds for the outcomes home win, draw and away win. In addition we add the corresponding bookmaker to the mix.



In [8]:
def get_max_odds(df, colnames):
    """
    Input: football dataframe and colnames for the resulting dataframe
    Function to retrieve the highest odds from the dataframe for each observation (i.e. row)
    Returns: pandas dataframe with matchup (home & away team) and the highest odds for outcomes H, D, A with date as index
    """
    # Get odds
    max_odds = df.apply(np.max, axis=1)
    # Get Bookie
    bookies = df.idxmax(axis=1)
    
    max_odds_df = pd.concat([max_odds, bookies], axis=1)
    max_odds_df.columns = colnames
    
    return max_odds_df

In [9]:
max_H_odds = get_max_odds(df[df_odds.columns[2::3]], colnames=["H_MaxOdd", "H_MaxBookie"])
max_D_odds = get_max_odds(df[df_odds.columns[3::3]], colnames=["D_MaxOdd", "D_MaxBookie"])
max_A_odds = get_max_odds(df[df_odds.columns[4::3]], colnames=["A_MaxOdd", "A_MaxBookie"])

df_total = pd.concat([df,
                      max_H_odds,
                      max_D_odds,
                      max_A_odds], axis=1)

df_total.head()

,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,PSCH,PSCD,PSCA,Season,H_MaxOdd,H_MaxBookie,D_MaxOdd,D_MaxBookie,A_MaxOdd,A_MaxBookie
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-26,D1,Bayern Munich,Werder Bremen,6,0,H,2,0,H,27,...,1.08,14.50,34.00,2016/2017,1.15,IWH,12.20,PSD,27.00,PSA
2016-08-27,D1,Augsburg,Wolfsburg,0,2,A,0,1,A,13,...,2.99,3.39,2.56,2016/2017,2.85,PSH,3.43,PSD,2.64,PSA
2016-08-27,D1,Dortmund,Mainz,2,1,H,1,0,H,17,...,1.29,6.55,10.80,2016/2017,1.40,IWH,5.41,PSD,9.49,PSA
2016-08-27,D1,Ein Frankfurt,Schalke 04,1,0,H,1,0,H,11,...,3.30,3.53,2.31,2016/2017,3.40,B365H,3.64,PSD,2.22,PSA
2016-08-27,D1,FC Koln,Darmstadt,2,0,H,1,0,H,21,...,1.54,4.33,7.12,2016/2017,1.80,B365H,3.72,PSD,5.31,PSA


In [10]:
# Invert max odds and apply sum to calculate margin
max_odds = df_total[["H_MaxOdd", "D_MaxOdd", "A_MaxOdd"]].apply(np.reciprocal, axis=1).apply(np.sum, axis=1) # returns series
df_total = pd.concat([df_total, max_odds.rename("MaxOdd_Margin")], axis=1) # update df_total with new column

df_total.sample(5)

,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,PSCD,PSCA,Season,H_MaxOdd,H_MaxBookie,D_MaxOdd,D_MaxBookie,A_MaxOdd,A_MaxBookie,MaxOdd_Margin
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-31,D1,Hannover,Schalke 04,0,1,A,0,1,A,15,...,3.72,1.91,2018/2019,4.66,PSH,3.80,VCD,1.86,PSA,1.015385
2019-01-20,D1,Schalke 04,Wolfsburg,2,1,H,1,1,D,20,...,3.26,3.71,2018/2019,2.27,PSH,3.30,B365D,3.70,PSA,1.013829
2017-08-26,D1,Stuttgart,Mainz,1,0,H,0,0,D,17,...,3.56,2.99,2017/2018,2.32,PSH,3.56,PSD,3.25,PSA,1.019626
2017-11-03,D1,Ingolstadt,FC Koln,2,2,D,1,1,D,26,...,3.15,3.09,2016/2017,2.60,IWH,3.30,IWD,3.19,PSA,1.001125
2018-09-29,D1,Hoffenheim,RB Leipzig,1,2,A,0,0,D,13,...,3.89,3.05,2018/2019,2.10,BWH,4.02,PSD,3.46,PSA,1.013964


We can now do some statistics on the results. Let's check if there are any arbitrage opportunities.

In [11]:
abs_number = (df_total["MaxOdd_Margin"] < 1).sum()
rel_number = abs_number / len(df_total) * 100
print("Absolute Number of matches with arbitrage opportunity: {} of {} matches in total".format(abs_number, len(df_total)))
print("Realtive Number of matches with arbitrage opportunity: {:.2f} %".format(rel_number))


Absolute Number of matches with arbitrage opportunity: 205 of 918 matches in total
Realtive Number of matches with arbitrage opportunity: 22.33 %


**Take away:** We see that one in five games offered the opportunity of arbitrage. This means, if you are after risk-free money in the betting market, you should be customer by as many bookmakers as possible, since it seem to turn out profitable.

We should now look at the max and mean return we would've realized if you played all arbitrage games that were possible in the last 3 years of 1. Bundesliga.

In [12]:
use_cols = ["HomeTeam", "AwayTeam", "Season",
            "H_MaxOdd", "H_MaxBookie",
            "D_MaxOdd", "D_MaxBookie",
            "A_MaxOdd", "A_MaxBookie",
            "MaxOdd_Margin"]
df_total[use_cols].sort_values("MaxOdd_Margin").head(10)

,HomeTeam,AwayTeam,Season,H_MaxOdd,H_MaxBookie,D_MaxOdd,D_MaxBookie,A_MaxOdd,A_MaxBookie,MaxOdd_Margin
Date,,,,,,,,,,
2018-07-04,M'gladbach,Hertha,2017/2018,2.00,IWH,3.77,PSD,5.75,B365A,0.939165
2018-02-02,FC Koln,Dortmund,2017/2018,5.35,PSH,3.90,BWD,2.00,IWA,0.943326
2017-10-28,Bayern Munich,RB Leipzig,2017/2018,1.50,IWH,6.00,VCD,8.77,PSA,0.947358
2017-12-17,Hannover,Leverkusen,2017/2018,4.48,PSH,3.99,PSD,2.10,IWA,0.950031
2016-12-21,Bayern Munich,RB Leipzig,2016/2017,1.55,IWH,5.25,B365D,8.74,PSA,0.950054
2016-12-17,RB Leipzig,Hertha,2016/2017,1.80,IWH,4.09,PSD,6.31,PSA,0.958533
2018-03-03,Augsburg,Hoffenheim,2017/2018,3.00,PSH,3.41,PSD,3.00,IWA,0.959922
2018-08-24,Bayern Munich,Hoffenheim,2018/2019,1.30,IWH,8.00,WHD,15.00,VCA,0.960897
2018-03-31,Bayern Munich,Dortmund,2017/2018,1.50,IWH,5.59,PSD,8.52,PSA,0.962928


We will now calculate the return and sort the values of the new *Return* column in ascending order. We can look at the highest possible return using the `.tail` method of pandas. We will also show the column *FTR* which stands for *full-time-result*.

In [13]:
df_total["Return"] = (1-df_total["MaxOdd_Margin"])
df_total[["HomeTeam", "AwayTeam", "FTR", "Return"]].sort_values("Return").tail()

,HomeTeam,AwayTeam,FTR,Return
Date,,,,
2016-12-21,Bayern Munich,RB Leipzig,H,0.049946
2017-12-17,Hannover,Leverkusen,D,0.049969
2017-10-28,Bayern Munich,RB Leipzig,H,0.052642
2018-02-02,FC Koln,Dortmund,A,0.056674
2018-07-04,M'gladbach,Hertha,H,0.060835


**Take away:** The match with the highest arbitrage opportunity was possible in the match between *Borussia Mönchengladbach* and *Hertha BSC* in the season 2017/18. The possible return was around 6 percent. The distribution of your wager is given as


|_                    |Home Win     |Draw         |Away Win     |
|:--------------------|:-----------:|:-----------:|:-----------:|
|Distribution of wager|53.24%       |28.24%       |18.52%       |


Okay. We know that there is quite a large opportunity for arbitrage (i.e. one in five games). But is it *worth* it? We can define a *minimum margin* we want to realize. I would go for 5 percent.

In [14]:
min_margin = 0.05
match_count_minimum_return = df_total[df_total.Return > min_margin].shape[0]

print("We want a minimum arbitrage margin of {:.2f} percent.".format(min_margin*100))
print("There are exactly {} matches in the data set that meet this requirement.".format(match_count_minimum_return))
#print(df_total[df_total.Return > min_margin].Return)

We want a minimum arbitrage margin of 5.00 percent.
There are exactly 3 matches in the data set that meet this requirement.


In [16]:
# Save data for part two
#df_total.to_pickle("data/output/df_total_part1.pkl")

## Conclusion

We saw, that there was the possiblity for arbitrage in one fifths of the games in last three years of Bundesliga. This sounds quite high considering there are nine matches played each weekend. To calm ourselfs down, we saw that there is only a small fraction of games even worth considering. For my minimum margin of five percent, there were only 3 matches in total in the whole data set available. This is enough to conclude, that looking for arbitrage opportunities in the closing odds (i.e. the odds set by the bookmakers right before the match begins) is not worth the time invested.

This concludes part one of my data analysis for now. In part two we will look at the most famous betting strategies around. On is the famous *"Favourite Longshot-Bias"*, which is also a kind of proven concept in the scientific literature.